In [348]:
# %pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [1]:
# %pip install -qU langchain-openai

In [2]:
# %pip install langchain-chroma
# ollama serve
# ollama run thewindmom/llama3-med42-8b
# ollama run llama3.2

In [2]:
import pandas as pd
from openai import OpenAI
import re

In [3]:
import getpass
import os
import json
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


In [8]:
ner = pd.read_csv('data/ner_test_flat.tsv')

In [9]:
ner.shape

(1992, 4)

In [10]:
ner.sample(3)

,note_ID,text,ner_text,ner_type
912,59,History of Present Illness: 57-year-old male w...,aspirin,medication
1713,84,History of Present Illness: Mr. [ Known lastn...,albuterol,medication
311,23,History of Present Illness: 65 year-old gentle...,haloperidol lactate,medication


In [11]:
sym_med = []
for i in ner['note_ID']:
    symptoms = ner[(ner['note_ID']==i) & (ner['ner_type']=='symptom')]['ner_text']
    medications = ner[(ner['note_ID']==i) & (ner['ner_type']=='medication')]['ner_text']
    for m in medications:
        for s in symptoms:
            sym_med.append((s, m))

In [12]:
sym_med = list(set(sym_med))

In [13]:
len(sym_med)

3770

In [14]:
meds = ner[ner['ner_type']=='medication']['ner_text'].value_counts().head(10).index.tolist()

In [15]:
sysm = ner[ner['ner_type']=='symptom']['ner_text'].value_counts().head(10).index.tolist()

In [16]:
# meds+sysm

In [17]:
# entities = ner['ner_text'].unique()

In [18]:
# len(entities)

In [19]:
ents = meds+sysm

In [20]:
ents

['aspirin',
 'coumadin',
 'warfarin',
 'docusate sodium',
 'methadone',
 'tizanidine',
 'senna',
 'simvastatin',
 'omeprazole',
 'nitroglycerin',
 'chest pain',
 'fever',
 'shortness of breath',
 'sob',
 'orthopnea',
 'dysuria',
 'headache',
 'diarrhea',
 'cough',
 'constipation']

In [21]:
ner_pair = pd.read_csv('data/ner_test.csv')

In [22]:
ner_pair.head()

,note_ID,text,symptoms,medications
0,1,History of Present Illness:\n41 y/o M w/ h/o H...,"{'myalgias', 'hemoptysis', 'calf pain', 'tachy...","{'morphine', 'cefazolin', 'prilosec', 'senna',..."
1,8,History of Present Illness:\n64 y/o male w/kno...,{'pain'},"{'ranitidine hcl', 'aspirin', 'lipitor', 'ibup..."
2,9,"History of Present Illness:\nIn brief, this is...","{'tachycardic', 'myoclonic seizures', 'aphasic...","{'phenytoin', 'morphine', 'meropenem', 'morphi..."
3,10,"History of Present Illness:\nIn brief, this is...","{'tachycardic', 'myoclonic seizures', 'aphasic...","{'phenytoin', 'morphine', 'meropenem', 'morphi..."
4,11,History of Present Illness:\nHPI: This is a 69...,"{'headaches', 'lethargic', 'lethargy'}","{'thiamine hcl', 'aspirin', 'docusate sodium',..."


## Load pubmed data

In [23]:
file_path = 'data/pubmed.json'
# file_path = 'pubmed/2019-1-pubmed_articles.json'
with open(file_path, 'r', encoding='utf-8') as file:
    # Load JSON data from the file
    data = json.load(file)

In [24]:
len(data)

633712

In [25]:
pattern = '|'.join(map(re.escape, ents))

In [26]:
n = 0
samples = []
for d in data:
    # if n < 100000:
    if (d['Abstract']!='') and (re.search(pattern, d['Abstract'])):
        samples.append(d)
            # n = n + 1

In [27]:
# data = [d for d in data if re.search(pattern, d['Abstract'])]

In [28]:
# data = [d for d in data if d['Abstract']!='']

In [29]:
# samples[0]

In [30]:
len(samples)

20369

In [31]:
df = pd.DataFrame(samples)

In [32]:
print(df.shape)
df = df.loc[df['Abstract']!='']
print(df.shape)

(20369, 5)
(20369, 5)


In [33]:
# df.head()

In [34]:
# df.to_csv('data/2019-1-pubmed_articles.csv')
# df.to_csv('data/2019-1-pubmed_articles.csv')

In [35]:
df['Authors'] = df['Authors'].astype(str)
# df['combined'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [36]:
# df['Authors'] = 
# df['Authors'].iloc[0]
# df['Abstract'] = df[['PMID', 'Title', 'Journal', 'Authors', 'Abstract']].agg(' - '.join, axis=1)

In [37]:
list(df)

['PMID', 'Title', 'Journal', 'Authors', 'Abstract']

In [38]:
# df

In [39]:
# # pid = [55730, ]
# # docs = notes.loc[notes['subject_id']==pid]
# docs = notes.loc[notes['category']=='Discharge summary']

In [40]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.retrievers import BM25Retriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.output_parsers import PydanticOutputParser

In [41]:
# loader = CSVLoader(file_path='data/2019-1-pubmed_articles.csv',
#     csv_args={
#     # 'delimiter': ',',
#     # 'quotechar': '"',
#     'fieldnames': ['PMID', 'Title', 'Journal', 'Authors', 'Abstract']
# }
#     )

In [42]:
# docs = loader.load()

In [43]:
# from langchain.schema import Document

In [44]:
loader = DataFrameLoader(df, page_content_column="Abstract")
docs = loader.load()
# docs = [
#     Document(
#         page_content=str(d),
#     )
#     for d in data[:100]
# ]

In [45]:
# docs[3]

In [46]:
embeddings = OllamaEmbeddings(model="llama3.2")
llm = OllamaLLM(model="llama3.2")

In [47]:
%%time
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
# vectordb.persist()
# Retrieve and generate using the relevant snippets of the blog.
# retriever = vectorstore.as_retriever()

retriever = BM25Retriever.from_documents(splits, k=10)

CPU times: user 4.58 s, sys: 248 ms, total: 4.83 s
Wall time: 4.82 s


In [48]:
# %pip install rank_bm25

In [49]:
# from langchain.retrievers.ensemble import EnsembleRetriever
# from langchain.retrievers import BM25Retriever

# bm25_retriever = BM25Retriever.from_documents(splits)
# retriever = EnsembleRetriever(retrievers=[bm25_retriever, vectorstore.as_retriever()], weights=[0.5, 0.5])

In [50]:
# retriever = vectorstore.as_retriever(
#     search_type="similarity",  # Options: "similarity", "mmr", "similarity_score_threshold"
#     search_kwargs={"k": 10}  # Number of top results to retrieve
# )

In [51]:
# vectordb = Chroma(persist_directory='data/mimic_embs', embedding_function=embeddings)

In [69]:
from pydantic import BaseModel
from typing import Union

# Define a Pydantic model for structured JSON output
class AdverseEffectResponse(BaseModel):
    PMID: Union[int, str]  # Accepts both int and "N/A"
    evidence: str  # Summary of evidence

parser = PydanticOutputParser(pydantic_object=AdverseEffectResponse)

In [70]:
# prompt = hub.pull("rlm/rag-prompt")
prompt = PromptTemplate.from_template("You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. \nContext: {context} \nQuestion: {question} \nAnswer:")

def format_docs(docs):
    return "\n\n".join(
        f"Content: {doc.page_content}\nMetadata: {doc.metadata}" for doc in docs
    )
    # return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    | parser
)

In [71]:
# rag_chain.invoke('what is the median red blood cell content was?')

In [72]:
# df.head(2)

In [108]:
def rag_pair(medication, sysm):
    question = f"""
    Determine whether {medication} has been associated with {sysm} as a potential adverse effect. 
    If evidence exists, **return only a JSON object** in this format:
    {{
    "PMID": <PubMed ID>,
    "evidence": "<summary of evidence>"
    }}
    
    If no evidence exists, return:
    
    {{
        "PMID": "N/A",
        "evidence": "No evidence found."
    }}
    """
    response = rag_chain.invoke(question)
    print(response)

In [109]:
rag_pair('aspirin', 'hemorrhage')

PMID='32830272' evidence='Aspirin of potential value in patients with autoimmune systemic rheumatic diseases due to increased risk of cardiovascular complications, which makes aspirin a potential treatment option for these subjects'


In [110]:
rag_pair('aspirin', 'bleeding')

PMID='37132104' evidence='Aspirin confers a net benefit in patients with CAC ≥100 and ASCVD risk ≥5% but would experience a net harm in patients with a high bleeding risk, irrespective of ASCVD risk or CAC.'


In [78]:
from tqdm import tqdm

In [80]:
results = {}

In [ ]:
# https://pubmed.ncbi.nlm.nih.gov/8824810/

# hemorrhage
# # bleeding 

In [81]:
for sysm, medication in tqdm(sym_med): 
    key = sysm+'-'+medication
    if key not in results:
        # medication = "aspirin"
        # sysm = 'hemorrhage'
        # Using f-string to format the string with the variables
        
        question = f"""
        Determine whether {medication} has been associated with {sysm} as a potential adverse effect. 
        If evidence exists, **return only a JSON object** in this format:
        {{
        "PMID": <PubMed ID>,
        "evidence": "<summary of evidence>"
        }}
        
        If no evidence exists, return:
        
        {{
            "PMID": "N/A",
            "evidence": "No evidence found."
        }}
        """
        # Print the correctly assigned string
        # print(question)
        try:
            response = rag_chain.invoke(question)
        except Exception as e:
            if "OutputParserException" in str(e):
                response = rag_chain.invoke(question)
        
        results[key] = {'symptom': sysm,
                        'medication': medication,
                        'PMID': response.PMID,
                        'evidence': response.evidence
        }

100%|██████████| 3770/3770 [1:48:51<00:00,  1.73s/it]  


In [83]:
len(results)

3770

In [142]:
rets = {}

In [145]:
for sysm, medication in tqdm(sym_med): 
    key = sysm+'-'+medication
    if key not in rets:
        # medication = "aspirin"
        # sysm = 'hemorrhage'
        # Using f-string to format the string with the variables
        
        question = f"""
        Determine whether {medication} has been associated with {sysm} as a potential adverse effect. 
        If evidence exists, **return only a JSON object** in this format:
        {{
        "PMID": <PubMed ID>,
        "evidence": "<summary of evidence>"
        }}
        
        If no evidence exists, return:
        
        {{
            "PMID": "N/A",
            "evidence": "No evidence found."
        }}
        """
        # Print the correctly assigned string
        # print(question)
        retrieved_docs = retriever.invoke(question)

        # print(len(retrieved_docs))

        # context = "\n\n".join(doc.page_content for doc in retrieved_docs)
        context = "\n\n".join(
                f"Content: {doc.page_content}\nMetadata: {doc.metadata}" for doc in retrieved_docs)
        # print(context)
        
        rets[key] = {'symptom': sysm,
                        'medication': medication,
                        'context': context
        }

100%|██████████| 3770/3770 [34:02<00:00,  1.85it/s]


In [146]:
# rets

In [ ]:
import pickle

In [149]:
# with open("rag_retrievals.pkl", "wb") as file:
#     pickle.dump(rets, file)

In [147]:
# with open("rag_results.pkl", "wb") as file:
#     pickle.dump(results, file)

In [ ]:
with open("rag_retrievals.pkl", "rb") as file:
    rets = pickle.load(file)

In [ ]:
with open("rag_results.pkl", "rb") as file:
    results = pickle.load(file)

In [89]:
results['diaphoretic-paroxetine hcl']

{'symptom': 'diaphoretic',
 'medication': 'paroxetine hcl',
 'PMID': '31361372',
 'evidence': "Paroxetine hcl has been associated with diaphoretic as a potential adverse effect, which is mentioned in the article 'Chemoprevention of colorectal cancer: Past, present, and future.'"}

In [153]:
dat = {}
for k in rets.keys():
    v = rets[k]
    sysm = dat.get(v['medication'], {})
    sysm[v['symptom']] = v['context']
    dat[v['medication']] = sysm

In [154]:
with open("rag-retrievals.json", "w") as file:
    json.dump(dat, file, indent=4)  # Save JSON to a file

In [104]:
dat = {}
for k in results.keys():
    v = results[k]
    sysm = dat.get(v['medication'], {})
    pids = sysm.get(v['symptom'], {})
    pid = (v['PMID'])
    if pid.isdigit():
        pids["PubMed ID: "+str(pid)] = v['evidence'] + " <a href="+f"https://pubmed.ncbi.nlm.nih.gov/{pid}/" +'target="_blank">Click here</a> '
    else:
        pids[pid] = v['evidence']
    sysm[v['symptom']] = pids
    dat[v['medication']] = sysm

In [105]:
# dat

In [106]:
with open("rag-data.json", "w") as file:
    json.dump(dat, file, indent=4)  # Save JSON to a file

In [97]:
# with open("rag_results.pkl", "rb") as file:
#     loaded_data = pickle.load(file)

In [140]:
# question = 'which study mentioned the median red blood cell content? Provide PMID of the study'
# question = 'What are the potential adverse effects of aspirin?'
# rag_chain.invoke(question)

# question = 'Is hemorrhage a potential adverse effect of aspirin? If so, provide evidence such as paper PMID and title in the context.'

AdverseEffectResponse(PMID=32162010, evidence='Yes, patients with cerebral amyloid angiopathy may experience a high risk of recurrent intracerebral hemorrhage (ICH) when treated with anticoagulation or platelet inhibition.')

In [66]:
df.loc[df['PMID']=='34908882']['Abstract'].tolist()

['INTRODUCTION: Pharmacologic therapy options for COVID-19 should include antiviral, anti-inflammatory, and anticoagulant agents. With the limited effectiveness, currently available virus-directed therapies may have a substantial impact on global health due to continued reports of mutant variants affecting repeated waves of COVID-19 around the world.\nMETHODS: We searched articles pertaining to aspirin, COVID-19, acute lung injury and pharmacology in PubMed and provide a comprehensive appraisal of potential use of aspirin in the management of patients with COVID-19. The scope of this article is to provide an overview of the rationale and currently available clinical evidence that supports aspirin as an effective therapeutic option in COVID-19.\nRESULTS: Experimental and clinical evidence are available for the potential use of aspirin in patients with COVID-19.\nDISCUSSION: Aspirin targets the intracellular signaling pathway that is essential for viral replication, and resultant inflamm

In [128]:
medication = 'aspirin'
symptom = 'hemorrhage'
question = f"""
    Determine whether {medication} has been associated with {symptom} as a potential adverse effect. 
    If evidence exists, **return only a JSON object** in this format:
    {{
    "PMID": <PubMed ID>,
    "evidence": "<summary of evidence>"
    }}
    
    If no evidence exists, return:
    
    {{
        "PMID": "N/A",
        "evidence": "No evidence found."
    }}
    """

In [129]:
print(question)


    Determine whether aspirin has been associated with hemorrhage as a potential adverse effect. 
    If evidence exists, **return only a JSON object** in this format:
    {
    "PMID": <PubMed ID>,
    "evidence": "<summary of evidence>"
    }
    
    If no evidence exists, return:
    
    {
        "PMID": "N/A",
        "evidence": "No evidence found."
    }
    


In [130]:
# question = 'Is Asthma an adverse drug reaction of PGE2?'
retrieved_docs = retriever.invoke(question)

print(len(retrieved_docs))

# context = "\n\n".join(doc.page_content for doc in retrieved_docs)
context = "\n\n".join(
        f"Content: {doc.page_content}\nMetadata: {doc.metadata}" for doc in retrieved_docs)
# print(context)

# prompt = hub.pull("rlm/rag-prompt")
prompt = PromptTemplate.from_template("You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nContext: {context} \nQuestion: {question} \nAnswer:")

example_messages = prompt.invoke(
    {"context": context, "question": question}
)

10


In [131]:
print(context)

Content: BACKGROUND: Kommerell's aneurysm is a saccular or fusiform dilatation found in 3-8% of Kommerell's diverticulum cases. A non-dissecting rupture rate of 6% has been reported. If ruptured, emergent surgical correction is usually granted. However, evidence regarding the optimal surgical approach in this acute setting is scarce. In this case report series, we aim to describe our experience managing type-1 non-dissecting ruptured Kommerell's aneurysm with hybrid emergent surgical approaches.
Metadata: {'PMID': '36964599', 'Title': "Emergent hybrid surgical approaches for non-dissecting ruptured Kommerell's aneurysm: a case report series.", 'Journal': 'Journal of cardiothoracic surgery', 'Authors': "['Alejandro Velandia-Sánchez', 'Sebastián Gómez-Galán', 'Sebastian Gallo-Bernal', 'Camilo A Polania-Sandoval', 'Ivonne G Pineda-Rodríguez', 'Paula Florez-Amaya', 'Lina M Sanabria-Arévalo', 'Julián Senosiain-González', 'Juan G Barrera-Carvajal', 'Juan P Umana', 'Jaime Camacho-Mackenzie']"

In [133]:
# print(example_messages)

In [135]:
chain = llm | StrOutputParser()
print(chain.invoke(example_messages))

{
    "PMID": "32830272",
    "evidence": "Aspirin may have potential value in patients with autoimmune systemic rheumatic diseases due to its cardiovascular benefits, but no specific mention of hemorrhage as an adverse effect is noted."
}
{
    "PMID": "36109476",
    "evidence": "No direct evidence linking aspirin use to hemorrhage is mentioned in the provided review on cannabidiol's therapeutic potential for aneurysmal subarachnoid hemorrhage."
}
{
    "PMID": "36085017",
    "evidence": "The case report mentions that anticoagulation with oral warfarin resolved a patient's frank haematuria, indicating that aspirin may not be directly associated with an increased risk of hemorrhage."
}


In [306]:
# df.head()

In [73]:
# # Cardiovascular Disease
# df[df['Title'].str.contains('asthma')]
df.loc[df['PMID']=='33392889']

,PMID,Title,Journal,Authors,Abstract
3845,33392889,A maternal death due to the intracerebral hemo...,Clinical rheumatology,"['Bijan Keikhaei', 'Najmieh Saadati', 'Mohamma...",Antiphospholipid syndrome (APS) is a systemic ...


In [80]:
# df[df['PMID']=='30635468']

In [ ]:
q = "Provide the titles and PMIDs of papers about Asthma"
retrieved_docs = retriever.invoke(q)

print(len(retrieved_docs))

context = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [102]:
from langchain.document_loaders import DataFrameLoader

# Sample DataFrame
data = {
    "Title": ["Paper1", "Paper2", "Paper3"],
    "Abstract": ["Abstract1", "Abstract2", "Abstract3"],
    "Year": [2021, 2022, 2023],
    "Source": ["Journal1", "Journal2", "Journal3"]
}
df = pd.DataFrame(data)

# Load documents with metadata
loader = DataFrameLoader(
    df.head(100),  # Use the first 100 rows
    page_content_column="Abstract",  # Use the "Abstract" column as the main content
    # metadata_columns=["Title", "Year", "Source"]  # Include these columns as metadata
)

docs = loader.load()

# Inspect the documents
for doc in docs:
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)
    print()

Content: Abstract1
Metadata: {'Title': 'Paper1', 'Year': 2021, 'Source': 'Journal1'}

Content: Abstract2
Metadata: {'Title': 'Paper2', 'Year': 2022, 'Source': 'Journal2'}

Content: Abstract3
Metadata: {'Title': 'Paper3', 'Year': 2023, 'Source': 'Journal3'}

